In [0]:
# %load_ext autoreload
# %autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0


# Tables

---

## S3 stream source
- clickstream

## ingested tables
- clickstream_raw

## bronze tables
- clickstream_bronze

In [0]:
from config.settings import *

In [0]:
%run "../../config/setup"

In [0]:

streaming_df = (spark.readStream \
    .format("cloudFiles") \
    .option("cloudFiles.format", "json") \
    .option("path", f"{S3_BUCKET_URI}/clickstreams")
    .option("cloudFiles.schemaLocation", f"{S3_BUCKET_URI}/schema")
    .load())
    

In [0]:
streaming_df.printSchema()

In [0]:

query = (streaming_df
    .writeStream \
    .format("delta") \
    .option("checkpointLocation", f"{S3_BUCKET_URI}/checkpoints") \
    .outputMode("append") \
    .trigger(availableNow=True) \
    .toTable("clickstream_raw"))

query.awaitTermination()

In [0]:
%sql

GRANT SELECT ON TABLE clickstream_raw TO devs;

In [0]:
(spark
    .table("clickstream_raw")
    .drop("_rescued_data")
    .write
    .mode("overwrite")
    .saveAsTable("clickstream_bronze"))

In [0]:
%sql
GRANT SELECT ON TABLE clickstream_bronze TO devs;